# What is Micrograd?

"**Micrograd**" is small autograd engine. The word **Autograd** is short for "**Automatic Gradient**". Autograd is also the name of PyTorch's automatic differentiation system.

What "Micrograd" does, is that it implements the "Backpropagation" algorithm. "Backprop" helps us **efficiently compute the gradient of a loss/error function with respect to the weights of a given neural network**.

With the gradient, we can iterativelly tune/tweak the weights of a network to minimize our loss function.

I am building Micrograd following **Andrej Karpathy** [YouTube](https://youtu.be/VMj-3S1tku0) video.

# Why Am I learning this? 💡🧠

1. I want **Deep understanding**. As a newcomer, I do not want to fall into the rush of building the "cool" stuff. I would like to practice patience, taking the time to learn... I want to believe that it will pay ten fold in the future.

2. According to Andrej, **Backpropagation is at the CORE of any modern deep learning frameworks**, like PyTorch, JAX, TensorFlow, etc... I believe him.

3. My supervisor told me that PyTorch's Autograd is useful when debugging a neural network, and debugging is an important part of neural network development. So, taking some time to understand what happens under the hood might pay in the future? We'll see. I can only connect the dots looking backward. 😉

# What can Micrograd do?

